In [1]:
# =====================================================================
#  Supplementary sweep (7⁵ grid) – joint variation of
#       (δ₀ , p₁ , p₂ , a , b)
#  -------------------------------------------------------------------
#  Purpose
#  -------
#  Re-run all three key empirical checks on a denser 5-parameter grid
#  that now **also sweeps b** (in the main text b was held fixed):
#
#      • C1 : Steady state  ⇔  f(δ) has zeros              (5-class tally)
#      • C3 : Trajectory is monotone *before* the plateau  (inc / dec / non)
#      • C5 : Converges to the very first zero in its path (l / r / nl / nr)
#
#  Grid size & ranges
#  ------------------
#      7 points per axis  →  7⁵ = 16 807 trajectories
#
#      δ₀ ∈ linspace(−π,  π,  7)
#      p₁ ∈ linspace( 0,  2,  7)
#      p₂ ∈ linspace( 0,  2,  7)
#      a  ∈ linspace( 1,  5,  7)
#      b  ∈ linspace( 1,  5,  7)   ← **newly varied**
#
#  Numerical settings (identical to main-text scripts)
#  ---------------------------------------------------
#      • dt = 0.001,  T_MAX = 100
#      • Plateau test: 10-s window (10 000 pts), ±5 × 10⁻⁴ band
#      • C1 / C5 zero-match tolerance  = 1 × 10⁻² rad
#      • C3 monotone slack             = 1 × 10⁻⁶ rad
#      • Integrator tolerances         = rtol 1e-6, atol 1e-9
#
#  Output
#  ------
#      • “supplementary_analysis_vary_p1p2abdelta.xlsx” – full C1 table
#      • Console summaries for C1, C3, C5 (no extra files)
#
#  Implementation notes
#  --------------------
#      • The ODE is solved once per trajectory for C1, then solved
#        again for C3 (to extract the pre-plateau segment). C5 relies
#        only on the steady state from C1 and re-evaluates f(δ) on
#        fixed grids – no third integration is needed.
#      • Multiprocessing (Pool(cpu_count())) is used in all three parts
#        for speed; the logic and tolerances are the same as in the
#        single-core versions in the main text.
# =====================================================================

import numpy as np
import pandas as pd
from math import sin, cos, pi
from scipy.integrate import solve_ivp
import itertools, sys
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

# -------------------------- numerical constants -----------------------
T_MAX        = 100.0
DT           = 0.001
TIME_EVAL    = np.arange(0.0, T_MAX + DT, DT)

PLAT_WIN_S   = 10.0
PLAT_WIN_N   = int(PLAT_WIN_S / DT)        # 10-s window → 10 000 points
PLAT_TOL     = 5e-4                         # ± band for plateau
MATCH_TOL    = 1e-2                         # zero-match tolerance (C1/C5)
EPS_MONO     = 1e-6                         # slack for monotone test (C3)

# -------------------------- parameter grid ----------------------------
delta0_vals = np.linspace(-pi,  pi, 7)      # 7 values
p1_vals     = np.linspace(0.0, 2.0, 7)
p2_vals     = np.linspace(0.0, 2.0, 7)
a_vals      = np.linspace(1.0, 5.0, 7)
b_vals      = np.linspace(1.0, 5.0, 7)

GRID = list(itertools.product(delta0_vals, p1_vals, p2_vals, a_vals, b_vals))
assert len(GRID) == 16807, "Grid size should be 7^5 = 16 807"

# -------------------------- ODE & f(δ) --------------------------------
def single_cell_ode(t, y, p1, p2, a, b):
    θ, ψ = y
    s2, s, c = sin(2*(ψ-θ)), sin(ψ-θ), cos(ψ-θ)
    denom = 2*((a*s)**2 + (b*c)**2)**1.5
    num   = (a*b)*(a**2 - b**2)*s2
    dθ_dt = -1.0 - p1*num/denom
    dψ_dt = -p2*s2
    return [dθ_dt, dψ_dt]

def f_delta(δ, p1, p2, a, b):
    s2, s, c = np.sin(2*δ), np.sin(δ), np.cos(δ)
    denom = 2*((a*s)**2 + (b*c)**2)**1.5
    num   = (a*b)*(a**2 - b**2)*s2
    return -p2*s2 - (-1.0 - p1*num/denom)

# -------------------------- helpers -----------------------------------
def detect_plateau(δ):
    """Return (value, time) of first 10-s plateau, else (None,None)."""
    for i in range(len(δ) - PLAT_WIN_N):
        w = δ[i:i+PLAT_WIN_N]
        if np.max(np.abs(w - w[0])) < PLAT_TOL:
            return w[0], TIME_EVAL[i]
    return None, None

def zeros_one_period(p1, p2, a, b, n=20000):
    x = np.linspace(-pi, pi, n)
    y = f_delta(x, p1, p2, a, b)
    roots = []
    for i in range(n-1):
        if y[i]*y[i+1] < 0:
            lo, hi = x[i], x[i+1]
            for _ in range(30):
                mid = 0.5*(lo+hi)
                if f_delta(lo, p1, p2, a, b)*f_delta(mid, p1, p2, a, b) < 0:
                    hi = mid
                else:
                    lo = mid
            roots.append(0.5*(lo+hi))
    roots.sort()
    if not roots:
        return []
    start = roots[0]
    per = [z for z in roots if start < z < start+pi]
    end = start + pi
    if abs(end - per[-1]) > 1e-8:
        per.append(end)
    per.insert(0, start)
    return per

# -------------------------- C1 worker ---------------------------------
def worker_C1(params):
    δ0, p1, p2, a, b = params
    θ0, ψ0 = pi/3, pi/3 + δ0
    sol = solve_ivp(lambda t,y: single_cell_ode(t,y,p1,p2,a,b),
                    (0, T_MAX), [θ0, ψ0],
                    t_eval=TIME_EVAL,
                    rtol=1e-6, atol=1e-9)
    if not sol.success:
        return None   # discard rare failures

    δ = sol.y[1] - sol.y[0]
    steady_val, t_plateau = detect_plateau(δ)

    # zeros within one π-period centred on δ0
    zeros = zeros_one_period(p1, p2, a, b)
    has_zero = len(zeros) >= 2
    nearest_zero = None

    if has_zero:
        z_arr = np.asarray(zeros)
        nearest_zero = z_arr[np.argmin(np.abs(z_arr - δ0))]

    # --- category assignment -----------------------------------------
    if has_zero:
        if steady_val is not None:
            if abs(steady_val - nearest_zero) < MATCH_TOL:
                mtype = "(1)"
            else:
                mtype = "(2)"
        else:
            mtype = "(3)"
    else:
        if steady_val is None:
            mtype = "(4)"
        else:
            mtype = "(5)"

    return dict(delta0=δ0, p1=p1, p2=p2, a=a, b=b,
                steady_value=steady_val,
                steady_start_time=t_plateau,
                match_type=mtype)

# =====================================================================
#  PART 1 – C1  (grid sweep + Excel export)
# =====================================================================
print("► PART 1 (C1)  sweeping 16 807 trajectories …", flush=True)
with Pool(cpu_count()) as pool:
    rows = [r for r in tqdm(pool.imap_unordered(worker_C1, GRID),
                            total=len(GRID), desc="Integrating")
            if r is not None]

dfC1 = pd.DataFrame(rows)
dfC1.to_excel("supplementary_analysis_vary_p1p2abdelta.xlsx", index=False)

summary_C1 = dfC1["match_type"].value_counts().sort_index()
tot         = len(dfC1)
good        = summary_C1.get("(1)",0) + summary_C1.get("(4)",0)
print("\nC1 summary")
print("──────────")
print(summary_C1.to_string())
print(f"\nSatisfying C1 ((1)+(4)): {good}/{tot}  ({100*good/tot:.2f}%)")

# =====================================================================
#  PART 2 – C3  (monotonicity before plateau)
# =====================================================================
print("\n► PART 2 (C3)  pre-plateau monotonicity check …", flush=True)
df_C3 = dfC1[dfC1["match_type"].isin(["(1)","(2)"])].reset_index(drop=True)

def monotone(arr, eps=EPS_MONO):
    d = np.diff(arr)
    if np.all(d >= -eps):
        return "inc"
    if np.all(d <=  eps):
        return "dec"
    return "non"

def worker_C3(arg):
    (_, row) = arg
    δ0, p1, p2, a, b = row["delta0"], row["p1"], row["p2"], row["a"], row["b"]
    t_plateau = row["steady_start_time"]
    θ0, ψ0    = pi/3, pi/3 + δ0

    sol = solve_ivp(lambda t,y: single_cell_ode(t,y,p1,p2,a,b),
                    (0, T_MAX), [θ0, ψ0],
                    t_eval=TIME_EVAL,
                    rtol=1e-6, atol=1e-9)
    if not sol.success or np.isnan(t_plateau):
        return "solver_fail"

    δ = sol.y[1] - sol.y[0]
    segment = δ[TIME_EVAL < t_plateau]
    return monotone(segment)

with Pool(cpu_count()) as pool:
    flags = list(tqdm(pool.imap_unordered(worker_C3, df_C3.iterrows()),
                      total=len(df_C3), desc="Testing"))

df_C3["monotone"] = flags
monotab = df_C3["monotone"].value_counts().reindex(
            ["inc","dec","non","solver_fail"], fill_value=0)
print("\nMonotone-flag counts (pre-plateau):")
print(monotab.to_string())
ok_rate = 100*(monotab["inc"] + monotab["dec"]) / len(df_C3)
print(f"\nOverall pass rate: {ok_rate:.2f}%")

# =====================================================================
#  PART 3 – C5  (“first-zero” termination)
# =====================================================================
print("\n► PART 3 (C5)  first-zero termination test …", flush=True)
df_C5 = df_C3.copy()  # same subset (match_type 1 or 2)

labels = []
for _, row in df_C5.iterrows():
    δ0, p1, p2, a, b = row["delta0"], row["p1"], row["p2"], row["a"], row["b"]
    δ_star           = row["steady_value"]

    # zeros left / right of δ0
    left_range  = np.linspace(δ0 - pi, δ0, 2000)
    right_range = np.linspace(δ0, δ0 + pi, 2000)

    def find_zeros(rr, side):
        fd = f_delta(rr, p1, p2, a, b)
        z = [(rr[i]+rr[i+1])/2 for i in range(len(fd)-1) if fd[i]*fd[i+1] < 0]
        return z

    zL = find_zeros(left_range, "L")
    zR = find_zeros(right_range,"R")

    nearest_left  = min(zL, key=lambda x: abs(x-δ0)) if zL else None
    nearest_right = min(zR, key=lambda x: abs(x-δ0)) if zR else None

    if nearest_right is not None and abs(δ_star - nearest_right) < MATCH_TOL:
        labels.append("(r)")
    elif nearest_left is not None and abs(δ_star - nearest_left) < MATCH_TOL:
        labels.append("(l)")
    elif nearest_left is not None and δ_star < δ0:
        labels.append("(nl)")
    elif nearest_right is not None and δ_star > δ0:
        labels.append("(nr)")
    else:
        labels.append("(?)")

df_C5["nearest_zero_match"] = labels
ct_C5 = df_C5["nearest_zero_match"].value_counts().sort_index()
sat   = ct_C5.get("(l)",0) + ct_C5.get("(r)",0)

print(f"\nTotal entries checked: {len(df_C5)}")
print(f"Satisfying C5 ((r)+(l)): {sat} ({100*sat/len(df_C5):.2f}%)\n")
print("Breakdown by nearest_zero_match type:")
print(ct_C5.to_string())
print("\nAll three supplementary checks complete.")


► PART 1 (C1)  sweeping 16 807 trajectories …


Integrating: 100%|██████████| 16807/16807 [09:06<00:00, 30.73it/s] 



C1 summary
──────────
match_type
(1)    5690
(2)    6098
(4)    5019

Satisfying C1 ((1)+(4)): 10709/16807  (63.72%)

► PART 2 (C3)  pre-plateau monotonicity check …


Testing: 100%|██████████| 11788/11788 [00:18<00:00, 648.45it/s]



Monotone-flag counts (pre-plateau):
monotone
inc            9268
dec            2520
non               0
solver_fail       0

Overall pass rate: 100.00%

► PART 3 (C5)  first-zero termination test …

Total entries checked: 11788
Satisfying C5 ((r)+(l)): 11788 (100.00%)

Breakdown by nearest_zero_match type:
nearest_zero_match
(l)    2520
(r)    9268

All three supplementary checks complete.
